# FAISS 벡터 스토어 및 Amazon Bedrock를 사용한 RAG 
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.

---

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww


# 1. Bedrock Client 생성

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2.1)

In [ ]:
from langchain.llms.bedrock import Bedrock

In [ ]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

## Embedding 모델 선택

In [ ]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [ ]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [ ]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

# 3. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [ ]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [ ]:
def preprocess_data(df):
    ldf = df.copy()

    df_index = ldf.drop(['no'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

### CSVLoader 로 문서 로딩

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [ ]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    source_column="Source",
    encoding="utf-8"
)
documents_fsi = loader.load()

In [ ]:
print(len(documents_fsi))

documents_fsi[0]
# documents_fsi[-1]

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [ ]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 0
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0
    

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

## 청킹 통계 및 내용 확인

In [ ]:
from utils.rag import show_context_used, show_chunk_stat
    
show_chunk_stat(docs)

In [ ]:
show_context_used(docs, limit=5)

# 4. FAISS 벡터 Indexer 생성



In [ ]:
%%time
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vectorstore_faiss_fsi = FAISS.from_documents(
    documents=docs,
    embedding=llm_emb
)

print(f"vectorstore_faiss_fsi: number of elements in the index={vectorstore_faiss_fsi.index.ntotal}::")
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_fsi)

## 인덱싱된 벡터 확인

In [ ]:
vectorstore_faiss_fsi.index_to_docstore_id
#index_.docstore._dict[:10]
dict(list(vectorstore_faiss_fsi.docstore._dict.items())[:5])

# 5.질문 및 답변

이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [ ]:
query = "타기관OTP 등록 방법 알려주세요"
answer = wrapper_store_faiss.query(question=query, llm=llm_text)
print_ww(answer)

# 6.사용자 정의 가능한 옵션
위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# prompt_template = """\n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 읃답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}

# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=vectorstore_faiss_fsi.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# query = "3년 경과 후에 해지환급금 얼마 받아요?"  
query = "타기관OTP 등록 방법 알려주세요"
result = qa({"query": query})
print_ww(result['result'])

In [ ]:
show_context_used(result['source_documents'])        